## Computational Physics 2 (WS23/24) – Warm-up exercise

**Deadline: 31.10.2023 at 23:59**

Group: Emmy Noether

Students: Janik Rausch (628334), Camilo Tello Breuer (), Ida Wöstheinreich ()

You will implement and test two algorithms: **conjugate gradient** and **power method**. We will see in a moment what they are useful for. Fill the notebook following the instructions.

### Initialization

Here we load the needed libraries and we initialize the random number generator. **Important**: when using a random number generator, the seed needs to be set only once, preferebly at the beginning of the program.

In [16]:
import LinearAlgebra
import Random

using LinearAlgebra

Random.seed!(12345)

Random.TaskLocalRNG()

### Positive-definite matrices

Both algorithms will deal with hermitian positive-definite matrices. Recall:

- Given a complex square matrix $A$, its hermitian conjugate $A^\dagger$ is defined as its transposed complex-conjugated, i.e. $(A^\dagger)_{ij} = (A_{ji})^*$.
- A complex square matrix $A$ is said to be hermitian if $A=A^\dagger$.
- An hermitian matrix $A$ is said to be positive-definite if all its eigenvalues are positive.

The following function generates and returns a random positive-definite matrix, along with its eigenvactors and eigenvalues.

In [17]:
# The function 'generate_positive_definite_matrix' contructs an NxN positive-definite matrix 'A',
# its matrix of eigenvectors and its eigenvalues.
#
# Input parameters:
#    N (integer)        : size of output matrix 'A'
#    kappa (double)     : condition number of the output matrix 'A'
#                         see https://en.wikipedia.org/wiki/Condition_number#Matrices
# Output values: (A, U, evalues)
#    A (np.matrix)      : positive-definite NxN matrix with condition number kappa
#    U (np.matrix)      : NxN unitary matrix; each column of 'U' is an eigenvector of 'A'
#    evalues (np.array) : N-component array with eigenvalues of 'A'

function generate_positive_definite_matrix(N::Int64;kappa::Float64=10.)
    @assert(N > 1 , "N=" * string(N) * " must be an integer >= 2")
    @assert(kappa > 0. , "kappa=" * str(kappa) * " must be a positive float")
    
    rmat = Random.randn(N,N)+1im*Random.randn(N,N)
    U , _ = qr(rmat)
    evalues = vcat(1. .+ kappa*Random.rand(N-2),[1.;kappa])
    A = U*Diagonal(evalues)*U'
    
    return A, U, evalues
end

generate_positive_definite_matrix (generic function with 1 method)

### Power method

Given a positive-definite matrix $A$, the power method allows to approximate its largest eigenvalue and the corresponding eigenvector with a certain specified tolerance $\epsilon$. It is an iterative method: a number of steps are repeated cyclically, at each iteration one gets a better approximation of the eigenvalue and eigenvectors, the iteration is stopped when the approximation is good enough. It works as follows:

1. Generate a random complex vector $v$ with norm equal to 1.
2. Calculate $w=Av$ and $\mu = \| w \|$.
3. If $\| w - \mu v \| < \epsilon$, stop iteration and returns $\mu$ and $v$ are eigenvalue and eigenvector.
4. Replace $v \leftarrow \mu^{-1} w$ and repeat from 2.

**Task:** Implement the power method within the function ```power_method```, with the following specifications.

The *vector* $v$ is not necessarily a one-dimensional array, we want the flexibility to use more abstract vector spaces whose elements are generic $d$-dimensional arrays. In practice, the *vectors* $v$ must be implemented as ```numpy.ndarrays```. In this setup, the squared norm $\|v\|^2$ of the *vector* $v$ is given by the sum of the squared absolute value of all elements of $v$. Moreover, the *matrix* $A$ really needs to be thought as a linear function acting on the elements of the abstract vector space.

```power_method``` must be a function that takes three inputs:
- ```vshape``` is the shape of the elements $v$ of the abstract vector space;
- ```apply_A``` is a function that takes the vector $v$ (represented as an instance of ```numpy.ndarrays```) and returns the vector $Av$ (represented as an instance of ```numpy.ndarrays``` with the same shape as $v$);
- ```epsilon``` is the tolerance.

```power_method``` must return:
- the largest eignevalue $\mu$;
- the corresponding eigenvector (represented as an instance of ```numpy.ndarrays``` with the same shape as the input of ```apply_A```);
- the number of iterations.

A test function is provided below. Your implementation of ```power_method``` needs to pass this test.

In [18]:
# The function 'power_method' calculates an approximation of the largest eigenvalue 'mu'
# and corresponding eigenvector 'v' of the positive-definite linear map 'A'. The quality
# of the approximation is dictated by the tolerance 'epsilon', in the sense that the
# approximated eigenvalue and eigenvector satisfy
#   | A(v) - mu*v | < epsilon
#
# The vectors over which 'A' acts are generically d-dimensional arrays. More precisely,
# they are instances of 'numpy.ndarray' with shape 'vshape'.
#
# The linear map 'A' is indirectly provided as a function 'apply_A' which takes a vector
# v and returns the vector A(v).
#
# Input parameters of power_method:
#    vshape (tuple of ints) : shape of the arrays over which 'A' acts
#    apply_A (function)     : function v -> A(v)
#    epsilon (float)        : tolerance
# Output values: (mu, v, niters)
#    mu (float)             : largest eigenvalue of A
#    v (numpy.ndarray)      : corresponding eigenvector
#    niters (int)           : number of iterations

function power_method(vshape::Tuple{Vararg{Int}},apply_A::Function,epsilon::Float64)
    
    ### Implement your function here

    v = rand(Float64,vshape)
    niters = 0; maxiters = 1e5

    while niters < maxiters
        w = apply_A(v)
        global mu = norm(w)
        if norm(w-mu*v) < epsilon break end
        v = w/mu
        niters += 1
    end
    
    return mu, v, niters
end

power_method (generic function with 1 method)

#### Test

Run the following cell. If the power method is correctly implemented, then the test will pass.

In [19]:
function test_power_method()

    function test_engine(shape::Tuple{Vararg{Int}},epsilon::Float64)
        
        N = prod(shape)
        A , _ , _ = generate_positive_definite_matrix(N)
        
        function apply_A(v::Array)
            @assert size(v)==shape
            return reshape(A*reshape(v,(N,)),shape)
        end
        
        mu , v , niters = power_method(shape,apply_A,epsilon)
        delta = apply_A(v) - mu*v
        res = norm(delta)
        println("shape = " , shape , "\tresidue = " , res , "\titerations = " , niters , "\tTest passes: " , res<=epsilon)
    end
    
    
    test_engine((4,),1.e-8)
    test_engine((1,5),1.e-12)
    test_engine((3,2,4),1.e-8)
    test_engine((5,2),1.e-12)
end

test_power_method()

shape = (4,)	residue = 4.989364942362995e-9	iterations = 19	Test passes: true
shape = (1, 5)	residue = 9.98992743409436e-13	iterations = 366	Test passes: true
shape = (3, 2, 4)	residue = 9.412437181408594e-9	iterations = 303	Test passes: true
shape = (5, 2)	residue = 9.058975321632323e-13	iterations = 122	Test passes: true


### Conjugate gradient

**Task.**
1. Read about the conjugate gradient on Wikipedia.
2. Implement the conjugate gradient, using same conventions as for the power method.
3. Write a description of the algorithm here (in the same spirit as the description of the power method).
4. Run and pass the test provided below.
5. Discuss intermediate steps with tutors.

**Description.**
The conjugate gradient method allows solving the linear system $Ax=b$ in the case of a hermitian, positive-definite matrix $A$. It uses the fact that the solution $x$ is also the unique minimizer of $f(y)=y^\dag Ay-2y^\dag b$. The idea is to use gradient descent with the added constraint that subsequent search directions $p_i$ must be conjugate wrt. $A$, meaning $p_i^\dag Ap_j=0$ for $i\neq j$.

This leads to the following algorithm:

1. Start with an initial guess $x_0$ and the search direction $p_0=r_0=b-Ax_0$.
2. The next guess is $$x_{k+1}=x_k+\alpha_k p_k\quad\text{with}\quad\alpha_k=\frac{p_k^\dag r_k}{p_k^\dag Ap_k}.$$
3. The new residual is $r_{k+1}=b-Ax_{k+1}$. If $||r_{k+1}||\leq\epsilon||b||$, break the loop.
4. The new conjugate search direction is $$p_{k+1}=r_{k+1}-\sum_{i\leq k}\frac{p_i^\dag Ar_{k+1}}{p_i^\dag Ap_i}.$$
5. Repeat from 2.

However, this can be simplified by exploiting that the residuals $r_i$ are pairwise orthogonal (which can be shown by complete induction). This yields modified expressions for $\alpha_k$ and $p_{k+1}$:

$$\alpha_k=\frac{r_k^\dag r_k}{p_k^\dag Ap_k}\quad\text{and}\quad p_{k+1}=r_{k+1}+\frac{r_{k+1}^\dag r_{k+1}}{r_k^\dag r_k}p_k$$

In [20]:
# The function 'conjugate_gradient' calculates an approximation 'x' of 'A^{-1}(b)', where
# 'A' is a positive-definite linear map 'A', and 'b' is a vector in the domain of 'A'.
# The quality of the approximation is dictated by the tolerance 'epsilon', in the sense
# that the following inequality is satisfied
#   | A(x) - b | <= epsilon |b|
#
# The vectors over which 'A' acts are generically d-dimensional arrays. More precisely,
# they are instances of 'numpy.ndarray' with shape 'vshape'.
#
# The linear map 'A' is indirectly provided as a function 'apply_A' which takes a vector
# v and returns the vector A(v).
#
# Input parameters of power_method:
#    apply_A (function)     : function v -> A(v)
#    b (numpy.ndarray)      : vector 'b'
#    epsilon (float)        : tolerance
# Output values: (x, niters)
#    x (numpy.ndarray)      : approximation of 'A^{-1}(b)'
#    niters (int)           : number of iterations

function conjugate_gradient(apply_A::Function,b::Array,epsilon::Float64)

    ### Implement your function here

    x = rand(Float64,size(b)); r = b - apply_A(x); p = r
    niters = 0; maxiters = 1e5
    
    while niters < maxiters
        x = x + dot(r,r) / dot(p,apply_A(p)) * p
        r_new = b - apply_A(x)
        if norm(r_new) < epsilon*norm(b) break end
        p = r_new + dot(r_new,r_new) / dot(r,r) * p
        r = r_new 
        niters += 1
    end
    
    return x, niters
end

conjugate_gradient (generic function with 1 method)

In [21]:
function test_conjugate_gradient()

    function test_engine(shape::Tuple{Vararg{Int}},epsilon::Float64)
        
        N = prod(shape)
        A , _ , _ = generate_positive_definite_matrix(N)
        b = Random.randn(shape)+1im*Random.randn(shape)
        
        function apply_A(v::Array)
            @assert size(v)==shape
            return reshape(A*reshape(v,(N,)),shape)
        end
        
        x , niters = conjugate_gradient(apply_A,b,epsilon)
        delta = apply_A(x) - b
        res = norm(delta)
        println("shape = " , shape , "\tresidue = " , res , "\titerations = " , niters , "\tTest passes: " , res<=epsilon*norm(b))
    end
    
    
    test_engine((4,),1.e-8)
    test_engine((1,5),1.e-12)
    test_engine((3,2,4),1.e-8)
    test_engine((5,2),1.e-12)
end

test_conjugate_gradient()

shape = (4,)	residue = 1.8576197769089357e-13	iterations = 3	Test passes: true
shape = (1, 5)	residue = 2.4581080623448042e-15	iterations = 4	Test passes: true
shape = (3, 2, 4)	residue = 1.6052642153633876e-8	iterations = 22	Test passes: true
shape = (5, 2)	residue = 2.2732477266632935e-15	iterations = 9	Test passes: true
